In [1]:
conda create -n env_pytorch python=3.6 --yes

Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.9.2
  latest version: 4.10.3

Please update conda by running

    $ conda update -n base conda



## Package Plan ##

  environment location: /srv/conda/envs/env_pytorch

  added / updated specs:
    - python=3.6


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    _libgcc_mutex-0.1          |      conda_forge           3 KB  conda-forge
    _openmp_mutex-4.5          |            1_gnu          22 KB  conda-forge
    ca-certificates-2021.10.8  |       ha878542_0         139 KB  conda-forge
    certifi-2016.9.26          |           py36_0         217 KB  conda-forge
    ld_impl_linux-64-2.36.1    |       hea4e1c9_2         667 KB  conda-forge
    libffi-3.4.2               |       h9c3ff4c_4          57 KB  conda-forge
    libgcc-ng-11.2.0           |      h1d223b6_11         887 KB  conda-fo

In [2]:
conda activate env_pytorch


CommandNotFoundError: Your shell has not been properly configured to use 'conda activate'.
To initialize your shell, run

    $ conda init <SHELL_NAME>

Currently supported shells are:
  - bash
  - fish
  - tcsh
  - xonsh
  - zsh
  - powershell

See 'conda init --help' for more information and options.

IMPORTANT: You may need to close and restart your shell after running 'conda init'.



Note: you may need to restart the kernel to use updated packages.


In [3]:
conda install pytorch-cpu torchvision -c pytorch

Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.9.2
  latest version: 4.10.3

Please update conda by running

    $ conda update -n base conda



## Package Plan ##

  environment location: /srv/conda/envs/notebook

  added / updated specs:
    - pytorch-cpu
    - torchvision


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    cudatoolkit-10.1.243       |       h036e899_9       428.2 MB  conda-forge
    intel-openmp-2021.3.0      |    h06a4308_3350         1.4 MB  defaults
    mkl-2021.3.0               |     h06a4308_520       141.2 MB  defaults
    ninja-1.10.2               |       h4bd325d_1         2.4 MB  conda-forge
    pytorch-1.4.0              |py3.6_cuda10.1.243_cudnn7.6.3_0       432.9 MB  pytorch
    pytorch-cpu-1.1.0          |      py3.6_cpu_0        53.5 MB  pytorch
    torchvision-0.5.0          |       py36_cu101         

In [4]:
pip install dscribe

     |████████████████████████████████| 143 kB 4.7 MB/s eta 0:00:01
  Using cached pybind11-2.8.0-py2.py3-none-any.whl (207 kB)
     |████████████████████████████████| 2.2 MB 64.9 MB/s eta 0:00:01     |███████                         | 481 kB 64.9 MB/s eta 0:00:01
     |████████████████████████████████| 77 kB 8.0 MB/s  eta 0:00:01
     |████████████████████████████████| 3.4 MB 67.1 MB/s eta 0:00:01
     |████████████████████████████████| 25.3 MB 47.6 MB/s eta 0:00:01
  Created wheel for dscribe: filename=dscribe-1.1.0-cp36-cp36m-linux_x86_64.whl size=4436425 sha256=4d6257c0fe89dffd511276ae6671fafa35a06e34388651dba45293fac9dfea33
  Stored in directory: /home/jovyan/.cache/pip/wheels/f1/c5/a6/bd2c7a0567cdf2a32df4c921f81a9f5e06468cf04a93ccb618
Successfully built dscribe
Note: you may need to restart the kernel to use updated packages.


In [5]:
conda install -c conda-forge rdkit

Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.9.2
  latest version: 4.10.3

Please update conda by running

    $ conda update -n base conda



## Package Plan ##

  environment location: /srv/conda/envs/notebook

  added / updated specs:
    - rdkit


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    boost-1.74.0               |   py36h355b2fd_3         340 KB  conda-forge
    boost-cpp-1.74.0           |       h312852a_4        16.3 MB  conda-forge
    bzip2-1.0.8                |       h7f98852_4         484 KB  conda-forge
    cairo-1.16.0               |    h6cf1ce9_1008         1.5 MB  conda-forge
    pixman-0.40.0              |       h36c2ea0_0         627 KB  conda-forge
    pycairo-1.20.1             |   py36h726f1de_0          77 KB  conda-forge
    rdkit-2021.03.5            |   py36hbacbb34_0        38.3 MB  conda-forge
    

In [1]:
import numpy as np
import torch

from dscribe.descriptors import SOAP
from ase.io import read
from ase import Atoms
from ase.build import molecule

In [7]:
import zipfile
with zipfile.ZipFile('./data/dsgdb9nsd.xyz.zip', 'r') as zip_ref:
    zip_ref.extractall('./data')


In [2]:
from sklearn.kernel_ridge import KernelRidge
from matplotlib import pyplot
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import r2_score
import sqlite3
import numpy as np
import csv
import math
from rdkit import Chem
from rdkit.Chem import MACCSkeys
from sklearn.model_selection import cross_validate

In [3]:
def parse_float(s: str) -> float:
    try:
        return float(s)
    except ValueError:
        base, power = s.split('*^')
        return float(base) * 10**float(power)

In [4]:
'''
xyz format
- line 1: number of atoms n
- line 2: scalar properties
- line 3, ..., n+1: element type, coordinates xyz, Mulliken partial charges on atoms
'''

# parse an xyz file, returns a result dictionary of a molecule
def parse_xyz(filename):
    num_atoms = 0
    scalar_properties = []
    atomic_symbols = []
    xyz = []
    charges = []
    smiles = ""
    with open(filename, 'r') as f:
        for line_num, line in enumerate(f):
            if line_num == 0:
                num_atoms = int(line)
            elif line_num == 1:
                scalar_properties = [parse_float(i) for i in line.split()[2:]]
            elif 2 <= line_num <= 1 + num_atoms:
                atom_symbol, x, y, z, charge = line.split()
                atomic_symbols.append(atom_symbol)
                xyz.append([parse_float(x), parse_float(y), parse_float(z)])
                charges.append(parse_float(charge))
            elif line_num == num_atoms + 3:
                smiles = str(line.split())

    result = {
        'num_atoms': num_atoms,
        'atomic_symbols': atomic_symbols,
        'pos': torch.tensor(xyz),
        'charges': np.array(charges),
        'smiles': smiles
    }
    return result

In [5]:

# given 1 <= start <= 133885 and 1 <= end <= 133885 return array of soap 
# vectors for molecules start, start+1, ..., end
def build_soap(start, end):
    soaps = []
    for i in range(start, end+1):

        # create molecule object
        file_name = "./data/dsgdb9nsd.xyz/dsgdb9nsd_" + \
            str(i).zfill(6) + ".xyz"
        molecule = parse_xyz(file_name)
        molecule_obj = Atoms(symbols=molecule["atomic_symbols"], positions=molecule["pos"])

        # set up soap descriptor
        species = set()
        species.update(molecule_obj.get_chemical_symbols())

        soap = SOAP(
            species=species,
            periodic=False,
            rcut=5,
            nmax=8,
            lmax=8,
            average="outer",
            sparse=False
        )
        feature_vector = soap.create(molecule_obj)
        soaps.append(feature_vector)
    return soaps

In [6]:
def build_soap2(start, end):
    molecules = []
    for i in range(start, end+1):
        # create molecule object
        file_name = "./data/dsgdb9nsd.xyz/dsgdb9nsd_" + \
            str(i).zfill(6) + ".xyz"
        molecule = parse_xyz(file_name)
        molecule_obj = Atoms(symbols=molecule["atomic_symbols"], positions=molecule["pos"])
        molecules.append(molecule_obj)
        
    # create SOAP object
    soap = SOAP(
        species=["C", "H", "O", "N", "F"],
        periodic=False,
        rcut=5,
        nmax=9,
        lmax=9,
        average="outer",
        sparse=False
    )
    
    feature_vector = soap.create(molecules)
    return feature_vector

In [7]:
def transform(filepath):
    # parse gap data from qm9
    u0 = {}
    with open(filepath, newline = '') as csvfile:
        reader = csv.DictReader(csvfile)
        for row in reader:
            u0[row['mol_id']] = float(row['u0'])
    u0_data = list(u0.values())[:10000]
    soaps = build_soap2(1, 10000)
    
    u0_train, u0_test = np.array(u0_data[:7500]), np.array(u0_data[7500:])
    soap_train, soap_test = np.array(soaps[:7500],dtype=object), np.array(soaps[7500:],dtype=object)
    return soap_train, soap_test, u0_train, u0_test

In [8]:
def create_model():
    model = KernelRidge(kernel='laplacian')
    soap_train, soap_test, u0_train, u0_test = transform("./data/qm9.csv")
    model.fit(soap_train, u0_train)
    predict = model.predict(soap_test)
    
    r_sqr = r2_score(u0_test, predict)
    rmse = math.sqrt(mean_squared_error(u0_test, predict))
    mae = mean_absolute_error(u0_test, predict)
    #print("gap test: ", gap_test)
    #print("gap predict: ", predict)
    print("r^2: ",r_sqr)
    print("rmse: ", rmse)
    print("mae: ", mae)

In [ ]:
if __name__ == "__main__":
    create_model()